# Overview

## Import Libraries

In [8]:
import os
import pandas as pd
from PIL import Image
import torch
import torch.nn as nn #neural network
import torchvision as tv
import torchvision.transforms as TF

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

## Set Cuda Device and Import Data

In [9]:
# set test device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
index_df = pd.read_csv("../../data/image_index.csv", index_col=0)
index_df.head()

,img,data_set,condition
0,..\..\data\extracted\chest_xray\test\NORMAL\IM...,0,0
1,..\..\data\extracted\chest_xray\test\NORMAL\IM...,0,0
2,..\..\data\extracted\chest_xray\test\NORMAL\IM...,0,0
3,..\..\data\extracted\chest_xray\test\NORMAL\IM...,0,0
4,..\..\data\extracted\chest_xray\test\NORMAL\IM...,0,0


In [4]:
test_df = index_df[index_df.data_set==0]
train_df = index_df[index_df.data_set==1]

In [11]:
test_image_path = test_df.img[0]

## First Linear Prototype

The issue with this model is that the output is `[1, 640, 2]` when the output should be a shape of `[2,]`. 

A flattening maybe required

In [108]:
class linear_prototype(nn.Module):
    def __init__(self, img_h, img_w):
        super().__init__()
        #define sizes here
        self.h = img_h
        self.w = img_w
        self.linear1 = nn.Linear(img_w, 320)
        self.relu1 = nn.ReLU()
        self.linear2 = nn.Linear(320, 2)
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, x):
        # preprocess the input image
        x = self.preprocess_image(x)
        #============ Layer1==============#
        x = self.linear1(x)
        x = self.relu1(x)
        #============Layer2==============#
        x = self.linear2(x)
        
        return self.softmax(x)
    
    ## update this to sequential?
    def preprocess_image(self, path):
        resizer = TF.Resize((self.h, self.w)) #define resizer per new_h and new_w
        im = tv.io.read_image(path).type(torch.float) #read image as pytorch float tensor
        im = resizer(im) #resize image
        normalizer = TF.Normalize(im.mean(), im.std()) #initialize normalizer
        return normalizer(im) # return normalized pytorch float tensor

In [109]:
proto_model = linear_prototype(640, 640)

In [110]:
proto_model

linear_prototype(
  (linear1): Linear(in_features=640, out_features=320, bias=True)
  (relu1): ReLU()
  (linear2): Linear(in_features=320, out_features=2, bias=True)
  (softmax): Softmax(dim=1)
)

In [112]:
y_pred = proto_model(test_image_path)

In [113]:
y_pred.shape

torch.Size([1, 640, 2])

# Second Linear Prototype with Flattened Tensor

In [165]:
class linear_prototype2(nn.Module):
    def __init__(self, img_h, img_w):
        super().__init__()
        #define sizes here
        self.h = img_h
        self.w = img_w
        self.longshape = img_h*img_w
        self.linear1 = nn.Linear(self.longshape, 320)
        self.relu1 = nn.ReLU()
        self.linear2 = nn.Linear(320, 2)
        self.softmax = nn.Softmax(dim=0)
    
    def forward(self, x):
        # preprocess the input image
        x = self.preprocess_image_flat(x)
        print("preprocess shape ", x.shape)
        #============ Layer1==============#
        x = self.linear1(x)
        x = self.relu1(x)
        print("layer1 shape ", x.shape)
        #============Layer2==============#
        x = self.linear2(x)
        print("layer2 shape ", x.shape)
        return self.softmax(x)
    
    ## update this to sequential?
    def preprocess_image_flat(self, path):
        resizer = TF.Resize((self.h, self.w)) #define resizer per new_h and new_w
        im = tv.io.read_image(path).type(torch.float) #read image as pytorch float tensor
        im = resizer(im) #resize image
        normalizer = TF.Normalize(im.mean(), im.std()) #initialize normalizer
        im = normalizer(im) # return normalized pytorch float tensor
        return torch.flatten(im)

In [166]:
lin = linear_prototype2(640, 640)

In [167]:
lin(test_image_path)

preprocess shape  torch.Size([409600])
layer1 shape  torch.Size([320])
layer2 shape  torch.Size([2])


tensor([0.4756, 0.5244], grad_fn=<SoftmaxBackward>)

In [ ]:
def recall_loss(yhat, y):
    